### Custom Exceptions

In [1]:
class TimeoutError(Exception):
    """Timeout exception"""

In [3]:
try:
    raise TimeoutError('timeout error')
except TimeoutError as ex:
    print(ex)

timeout error


In [5]:
import sys

try:
    raise TimeoutError('timeout error')
except:
    ex_type, ex, tb = sys.exc_info()
    print(ex_type, ex, tb)

<class '__main__.TimeoutError'> timeout error <traceback object at 0x7f7ecc06c980>


In [6]:
try:
    raise TimeoutError('timeout exception')
except Exception as ex:
    print(ex.args, ex.__traceback__)

('timeout exception',) <traceback object at 0x7f7ebd76e8c0>


In [7]:
class ReadOnlyError(AttributeError):
    """Indicates an attribute is rean-only"""

In [ ]:
try:
    raise ReadOnlyError('Account number is read-only', 'BA10001')
except ReadOnlyError as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-only', 'BA10001')


In [10]:
try:
    raise ReadOnlyError('Account number is read-only', 'BA10001')
except BaseException as ex:
    print(repr(ex))

ReadOnlyError('Account number is read-only', 'BA10001')


In [14]:
class WebScraperException(Exception):
    """Base exception for WebScraper"""

class HTTPException(WebScraperException):
    """General HTTP exception for WebScraper"""

class InvalidUrlException(HTTPException):
    """Indicates the url is invalid (dns lookup fail)"""

class TimeoutException(HTTPException):
    """Indicates a general timeout exception in http connectivity"""

class PingTimeoutException(TimeoutException):
    """Ping time out"""

class LoadTimeoutEception(TimeoutException):
    """Page load timeout"""

class ParserException(WebScraperException):
    """Geneeral page parsing exception"""

```
WebScraperException
    - HTTPException
        - InvalidUrlException
        - TimeoutException
            - PingTimeoutException
            - LoadTimeoutException
    - ParserException
```

In [16]:
try:
    raise PingTimeoutException('Ping to www ... timed out')
except WebScraperException as ex:
    print(repr(ex))

PingTimeoutException('Ping to www ... timed out')


In [17]:
class APIException(Exception):
    """Base API Exception"""

In [19]:
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""

class DBException(ApplicationException):
    """General database exception"""

class DBConnectionError(DBException):
    """Indicates an error connecting to database"""

class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""

class NotFoundError(ClientException):
    """Indicates resource was not found"""

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""

So we have this exception hierarchy:

```
APIException
   - ApplicationException (5xx errors)
       - DBException
           - DBConnectionError
   - ClientException
       - NotFoundError
       - NotAuthorizedError
```

In [20]:
class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type

In [21]:
def lookup_account_by_id(account_id):
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f'Account number {account_id} is invalid.')

    if account_id < 100:
        raise DBConnectionError('Permanent failure connecting to database')
    elif account_id < 200:
        raise NotAuthorizedError('User does not have permissions to read this account.')
    elif account_id < 300:
        raise NotFoundError('Account not found')
    else:
        return Account(account_id, 'Savings')

In [ ]:
from http import HTTPStatus

def get_account(account_id):
    try:
        account = lookup_account_by_id(l)